In [2]:
import requests
import json
import base64
import pandas as pd
import matplotlib as mt

#Requesting access to the spotify web API
client_id = '16fd6188f83242288e8af5e299bc66dd'
client_secret = '6abc3f8f1a714265a17ad6423066ec43'

auth_header_b64 = base64.b64encode(f'{client_id}:{client_secret}'.encode('ascii'))
auth_header = auth_header_b64.decode('ascii')

req_header =  {'Authorization' : f'Basic {auth_header}'}
payload = {"grant_type": "client_credentials"}
url = "https://accounts.spotify.com/api/token"
resp = requests.post(url, headers = req_header, data = payload)
token = resp.json()

token['access_token']

#Defining header with access token
header = {
"Accept": "application/json",
"Content-Type": "application/json",
"Authorization": f"Bearer {token['access_token']}"
}

#Get 2 ablbums from artist and return data frame with list of album ids and names
def albums(artist):
    oi = requests.get(f"https://api.spotify.com/v1/artists/{artist}/albums?limit=2", headers = header)
    oi = pd.DataFrame(oi.json()['items'])
    oi = oi[oi['album_group'] == 'album'] #exclude single tracks
    oi['artist_name'] = oi['artists'][0][0]['name'] #Fetching artist's name
    oi = oi[['id', 'name', 'release_date', 'artist_name']]
    return oi

#get all tracks from one album
def tracks_albums(album_id):
    penis = requests.get(f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=50", headers = header)
    raw = pd.DataFrame(penis.json()['items'])
    penis = raw[['track_number']]
    penis['track_id'] = raw['id']
    #Guardando o nome do album:
    penis['album_id'] = album_id
    return penis

#pega lista de musicas de cada album
def album_to_track(id_artista):
    album_dos_artistas = albums(id_artista)
    tracks = map(tracks_albums, album_dos_artistas['id'])
    oi = map(lambda dt : dt.merge(album_dos_artistas, left_on = 'album_id', right_on = 'id'), tracks)
    return pd.concat(oi).drop(columns = ['id'])

#Getting global audio analysis for a single track
def audio_analysis(track):
    oi = requests.get(f"https://api.spotify.com/v1/audio-features/{track}", headers = header).json()
    return oi
    oi = pd.DataFrame(oi)
    return oi

#Function receives an artist and returns all of his albuns with global analysis for each track within each album.
def describing_tracks(artist):
    tracks_artists = album_to_track(artist)
    tracks = map(audio_analysis, tracks_artists['track_id'])
    tracks = pd.DataFrame(tracks)
    oi = tracks.merge(tracks_artists, left_on = "id", right_on = 'track_id').drop(columns = ['id'])
    return oi.drop(columns = ['analysis_url', 'track_href', 'uri'])

############ DEFINING QUERY ##################
#Define query. Search words separated by %20. Limited to 5 artists per genre
def query_genre(genero):
    oi = requests.get(f"https://api.spotify.com/v1/search?q=genre:{genero}&type=artist&limit=5", headers = header).json()
    oi = pd.DataFrame(oi['artists']['items'])
    return oi

def get_albums_from_query(genero):
    query = query_genre(genero) #Complete query result
    artists = list(query['id']) #List of artist's ids
    concat_albums = [] #empty list which will receive complete audio analysis for each artist's albums
    for i in range(len(artists)):
        oi = describing_tracks(artists[i])
        generos = [query['genres'][i]]
        oi['list_of_genres'] = str(generos)
        concat_albums.append(oi)
    final = pd.concat(concat_albums)
    final['genre_defiened_by_query'] = genero #appending genres defined by query
    return final


########## STARTS SAMPLING #############

# DEFINED SEARCH WORD, TOOK 5 RANDOM ARTISTS FOR EACH GENRE
# THEN TOOK 2 ALBUMS FROM EACH ONE OF THESE ARTISTS# 

jazz = get_albums_from_query('bebop')
bossa_nova = get_albums_from_query('bossa')
edm = get_albums_from_query('edm')
axe = get_albums_from_query('axe')
rock = get_albums_from_query('rock')
pop = get_albums_from_query('pop')
reggae = get_albums_from_query('reggae')
country = get_albums_from_query('country')
blues = get_albums_from_query('blues')
final_sample = pd.concat([bossa_nova, bebop, edm, axe, rock, pop, reggae, country, blues])
final_sample.to_csv(r'C:/Users/Lenovo/Documents/GitHub/music_order/dados.csv')

c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyError: 'items'